In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from lightgbm import LGBMClassifier

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [15]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [16]:
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/base/train_category.csv"
data = pd.read_csv(data_path, low_memory=False)

test_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/base/test_category.csv"
test_data = pd.read_csv(test_data_path)

In [17]:
#keep_features = ['grade', 'term_months', 'total_pymnt', 'total_pymnt_inv','total_il_high_credit_limit', 'loan_amnt']
keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']

data_classification = data.copy()
test_classification = test_data.copy()

data_classification = data_classification.drop(columns = keep_features)
test_classification = test_classification.drop(columns = keep_features)

In [18]:
## data Encoding
Labelencoding_features = ['term_months', 'sub_grade']
onehot_features = ['debt_settlement_flag', 'home_ownership', 'purpose']

onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

label_encoders = {}
for col in Labelencoding_features:
    le = LabelEncoder()
    data_classification[col] = le.fit_transform(data_classification[col])
    label_encoders[col] = le

# One-Hot Encoding
onehot_encoded = onehot_encoder.fit_transform(data_classification[onehot_features])
onehot_encoded_df = pd.DataFrame(
    onehot_encoded, 
    columns=onehot_encoder.get_feature_names_out(onehot_features),
    index=data_classification.index
)

# Merge
data_classification.drop(columns=onehot_features, inplace=True)
data_classification = pd.concat([data_classification, onehot_encoded_df], axis=1)

print("✅ Encoding 완료! 결과 shape:", data_classification.shape)

✅ Encoding 완료! 결과 shape: (1116458, 35)


In [19]:
##Test data encoding
for col in Labelencoding_features:
    le = label_encoders[col]
    test_classification[col] = le.transform(test_classification[col])

# One-Hot Encoding (train에서 fit된 onehot_encoder 재사용)
onehot_encoded_test = onehot_encoder.transform(test_classification[onehot_features])
onehot_encoded_test_df = pd.DataFrame(
    onehot_encoded_test,
    columns=onehot_encoder.get_feature_names_out(onehot_features),
    index=test_classification.index
)

# Merge
test_classification.drop(columns=onehot_features, inplace=True)
test_classification = pd.concat([test_classification, onehot_encoded_test_df], axis=1)

print("✅ 테스트 데이터 인코딩 완료! 결과 shape:", test_classification.shape)

✅ 테스트 데이터 인코딩 완료! 결과 shape: (744306, 35)


In [20]:
def encode_features(df, label_encoders, onehot_encoder, label_cols, onehot_cols):
    df = df.copy()
    
    # Label Encoding
    for col in label_cols:
        le = label_encoders[col]
        df[col] = le.transform(df[col])

    # One-Hot Encoding
    onehot_encoded = onehot_encoder.transform(df[onehot_cols])
    onehot_df = pd.DataFrame(
        onehot_encoded, 
        columns=onehot_encoder.get_feature_names_out(onehot_cols),
        index=df.index
    )

    df.drop(columns=onehot_cols, inplace=True)
    df = pd.concat([df, onehot_df], axis=1)
    
    return df

In [21]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

def load_data_by_method(method):
    test_data = test_classification
    X_test = test_data.drop(columns='loan_status')
    y_test = test_data[['loan_status']]

    base_data = data_classification

    if method == 'Base':
        X_train = base_data.drop(columns='loan_status')
        y_train = base_data[['loan_status']]

    elif method == 'Undersampling':
        data_x = base_data.drop(columns='loan_status')
        data_y = base_data[['loan_status']]
        X_temp, _, y_temp, _ = train_test_split(data_x, data_y, test_size=0.13, random_state=42, stratify=data_y)
        rus = RandomUnderSampler(random_state=42)
        X_under, y_under = rus.fit_resample(X_temp, y_temp)
        X_train, y_train = X_under, y_under

    elif method == 'SMOTE-NC':
        smote_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/smotenc/smotenc_data.csv"
        smote_data = pd.read_csv(smote_path)
        drop_cols = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt', 'funded_amnt']
        smote_data = smote_data.drop(columns=drop_cols)
        smote_data = encode_features(smote_data, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        X_train = smote_data.drop(columns='loan_status')
        y_train = smote_data[['loan_status']]

    else:
        fake_base = {
            'table-gan': 'tablegan/tablegan.csv',
            'Smotified-tablegan': 'tablegan/smotified-tablegan.csv',
            'vae-tablegan': 'tablegan/vae-tablegan.csv',
            'ctgan': 'ctgan/ctgan.csv',
            'smotified-ctgan': 'ctgan/smotified-ctgan.csv',
            'vae-ctgan': 'ctgan/vae-ctgan.csv',
            'smotified-vae-ctgan': 'ctgan/smotified-vae-ctgan.csv',
            'ctabgan': 'ctabgan/ctabgan.csv',
            'smotified-ctabgan': 'ctabgan/smotified_ctabgan.csv',
            'vae-ctabgan': 'ctabgan/vae-ctabgan.csv',
            'smotified-vae-ctabgan': 'ctabgan/smotified-vae-ctabgan.csv'
        }

        if method not in fake_base:
            raise ValueError(f"Unknown method: {method}")

        fake_path = f"C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/{fake_base[method]}"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1  # 모든 fake는 default

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt', 'funded_amnt']
        fake = fake.drop(columns=keep_features)
        fake['term_months'] = fake['term_months'].apply(lambda x: 36 if abs(x - 36) < abs(x - 60) else 60)
        fake = encode_features(fake, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        base_x = base_data.drop(columns='loan_status')
        base_y = base_data[['loan_status']]
        X_base, _, y_base, _ = train_test_split(base_x, base_y, test_size=0.2, stratify=base_y, random_state=42)
        train_real = pd.concat([X_base, y_base], axis=1)
        train_total = pd.concat([train_real, fake])
        train_total = shuffle(train_total, random_state=42)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

    return X_train, y_train, X_test, y_test

In [22]:
summary_with_sharpe_top = []

summary_with_grade_top = []

def linear_portfolio_top(Method, top_10_indices):
    selected = test_data.loc[top_10_indices.index].copy()
    returns = (selected['total_pymnt_inv'] - selected['funded_amnt']) / selected['funded_amnt']
    
    avg_return = returns.mean()
    std_return = returns.std()
    sharpe_ratio = avg_return / std_return if std_return != 0 else np.nan

    summary_with_sharpe_top.append({
        'Method': Method,
        'Average Return': avg_return * 100,
        'Std Dev': std_return,
        'Sharpe Ratio': sharpe_ratio
    }) 

    A_count = len(selected[selected['grade'] == 'A'])
    B_count = len(selected[selected['grade'] == 'B'])
    C_count = len(selected[selected['grade'] == 'C'])
    D_count = len(selected[selected['grade'] == 'D'])
    E_count = len(selected[selected['grade'] == 'E'])
    F_G_count = len(selected[selected['grade'] == 'F']) + len(selected[selected['grade'] == 'G'])

    summary_with_grade_top.append({
        'Method': Method,
        'A': A_count,
        'B': B_count,
        'C': C_count,
        'D': D_count,
        'E': E_count,
        'F+G': F_G_count,
        'Total': len(selected)
    }) 

In [ ]:
##상위 10% 투자
drop2 = ['loan_status', 'return']

def select_fully_paid(y_pred, method):
    scaler = StandardScaler()
    reg = LinearRegression()

    fully_paid_indices = (y_pred == 0)
    test_regression = test_data[fully_paid_indices]
    test_regression['return'] = (test_regression['total_pymnt_inv'] - test_regression['funded_amnt'])/test_regression['funded_amnt']

    train_regression = data.copy()

    if method == 'Base':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']

        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        data_x = train_regression.drop(columns='return')
        data_y = train_regression[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)

        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))
        return method, top_10_percent
    
    elif method == 'Undersampling':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['loan_amnt'])/train_regression['funded_amnt']
        drop_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        X_train = train_regression.drop(columns='loan_status')
        y_train = train_regression[['loan_status']]

        undersampler = RandomUnderSampler(random_state=42)
        X_under, y_under = undersampler.fit_resample(X_train, y_train)

        data_under = pd.concat([pd.DataFrame(X_under, columns=X_train.columns), pd.DataFrame(y_under, columns=['loan_status'])], axis=1)

        data_x = data_under.drop(columns=drop2)  
        data_y = data_under[['return']]  

        test_x = test_regression.drop(columns = drop2)
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)

        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent
    
    elif method == 'SMOTE-NC':
        smote_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/smotenc/smotenc_data.csv"
        data_smote = pd.read_csv(smote_path)
        
        data_smote['return'] = (data_smote['total_pymnt_inv'] - data_smote['funded_amnt'])/data_smote['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']

        train_regression = data_smote.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        X_train = train_regression.drop(columns='return')
        y_train = train_regression[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(X_train)

        X_train = data_x_scaled
        #y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent
    
    elif method == 'table-gan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/tablegan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent
        
    elif method == 'Smotified-tablegan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/smotified-tablegan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'vae-tablegan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/vae-tablegan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'ctgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'smotified-ctgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/smotified-ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'vae-ctgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/vae-ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)

        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'smotified-vae-ctgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/smotified-vae-ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        
        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 

    elif method == 'ctabgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        
        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'smotified-ctabgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/smotified_ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        
        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 

    elif method == 'vae-ctabgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/vae-ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        
        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 
    
    elif method == 'smotified-vae-ctabgan':
        train_regression['return'] = (train_regression['total_pymnt_inv'] - train_regression['funded_amnt'])/train_regression['funded_amnt']
        drop_features = ['loan_status','grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        train_regression = train_regression.drop(columns=drop_features)
        test_regression = test_regression.drop(columns=drop_features)
        
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/smotified-vae-ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        fake_regression = fake.copy()

        fake_regression['return'] = (fake_regression['total_pymnt_inv'] - fake_regression['funded_amnt'])/fake_regression['funded_amnt']
        fake_regression = fake_regression.drop(columns=drop_features)
        fake_regression['term_months'] = fake_regression['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        
        train_regression = encode_features(train_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        test_regression = encode_features(test_regression, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        fake_regression = encode_features(fake_regression,label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        train_total = pd.concat([train_regression, fake_regression])

        data_x = train_total.drop(columns='return')
        data_y = train_total[['return']]

        test_x = test_regression.drop(columns = 'return')
        test_y = test_regression[['return']]

        data_x_scaled = scaler.fit_transform(data_x)

        X_train = data_x_scaled
        y_train = data_y

        test_x_scaled = scaler.transform(test_x)

        reg.fit(X_train, y_train)

        #print(reg.coef_)
        y_pred = reg.predict(test_x_scaled)

        print("RMSE:", mean_squared_error(test_y, y_pred, squared=False))
        print("R²:", r2_score(test_y, y_pred))

        test_regression['predicted_return'] = reg.predict(test_x_scaled)
        top_10_percent = test_regression.sort_values(by='predicted_return', ascending=False).head(int(len(test_regression)*0.1))

        return method, top_10_percent 

In [24]:
from sklearn.utils import shuffle

classification_metrics_summary = []

def return_evaluation(model, data_x, data_y, X_test, y_test, method):
    print(f"Preprocessing method : {method}")

    if method == 'Base':
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2,
                                                           random_state=42, stratify=data_y)
        model.fit(
        X_train, y_train
        )

        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)


    elif method == 'Undersampling':
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.13, random_state=42, stratify=data_y)
        undersampler = RandomUnderSampler(random_state=42)
        X_under, y_under = undersampler.fit_resample(X_train, y_train)

        data_under = pd.concat([pd.DataFrame(X_under, columns=data_x.columns), pd.DataFrame(y_under, columns=['loan_status'])], axis=1)
        X_train = data_under.drop(columns=['loan_status'])  # Feature (입력 데이터)
        y_train = data_under[['loan_status']]  # Target (타겟 변수)

        model.fit(
        X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)

    elif method == 'SMOTE-NC':
        smote_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/smotenc/smotenc_data.csv"
        data_smote = pd.read_csv(smote_path)

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        data_smote_classification = data_smote.copy()
        data_smote_classification = data_smote_classification.drop(columns = keep_features)

        data_smote_classification = encode_features(data_smote_classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        data_x = data_smote_classification.drop(columns='loan_status') 
        data_y = data_smote_classification[['loan_status']]

        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, random_state=42, stratify=data_y)

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'table-gan':
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/tablegan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1
        
        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)

        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, random_state=42, stratify=data_y)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total, random_state=42)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
        
    elif method == 'Smotified-tablegan':
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/smotified-tablegan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'vae-tablegan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/vae-tablegan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'ctgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'smotified-ctgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/smotified-ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'vae-ctgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/vae-ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
        
    elif method == 'smotified-vae-ctgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/smotified-vae-ctgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'ctabgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'smotified-ctabgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/smotified_ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'vae-ctabgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/vae-ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    elif method == 'smotified-vae-ctabgan':
        #Fake dataset
        fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/smotified-vae-ctabgan.csv"
        fake = pd.read_csv(fake_path)
        fake['loan_status'] = 1

        keep_features = ['grade', 'total_pymnt', 'total_pymnt_inv', 'loan_amnt','funded_amnt']
        fake_Classification = fake.copy()
        fake_Classification = fake_Classification.drop(columns = keep_features)
        fake_Classification['term_months'] = fake_Classification['term_months'].apply(lambda x:36 if abs(x-36)<abs(x-60) else 60)
        fake_Classification = encode_features(fake_Classification, label_encoders, onehot_encoder, Labelencoding_features, onehot_features)
        
        X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.2, stratify=data_y, random_state=42)
        train_dataset = pd.concat([X_train, y_train], axis=1)
        
        train_total = pd.concat([train_dataset,fake_Classification])
        train_total = shuffle(train_total)

        X_train = train_total.drop(columns='loan_status')
        y_train = train_total[['loan_status']]

        model.fit(
            X_train, y_train
        )
        y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

        threshold = 0.5
        y_pred = (y_prob >= threshold).astype(int)
    
    report = classification_report(
        y_test, y_pred,
        target_names=["Fully Paid", "Default"],
        output_dict=True,
        zero_division=0
    )

    classification_metrics_summary.append({
        "Method": method,
        "Class 0 Precision": report["Fully Paid"]["precision"],
        "Class 0 Recall": report["Fully Paid"]["recall"],
        "Class 0 F1": report["Fully Paid"]["f1-score"],
        "Class 1 Precision": report["Default"]["precision"],
        "Class 1 Recall": report["Default"]["recall"],
        "Class 1 F1": report["Default"]["f1-score"],
        "Macro Avg F1": report["macro avg"]["f1-score"],
        "Weighted Avg F1": report["weighted avg"]["f1-score"],
    })

    #10% index뽑기기
    Method, top_10_indices = select_fully_paid(y_pred, method)

    #portfolio 만들기들기
    linear_portfolio_top(Method, top_10_indices)

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
import numpy as np

class MLPClassifierTorch(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_dims=(32,), lr=0.001, epochs=10, batch_size=128, device='cuda'):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        # 모델 초기화
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.model = nn.Sequential(*layers).to(self.device)

        self.loss_fn = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def fit(self, X, y):
        X_np = X.values if hasattr(X, 'values') else X
        y_np = y.values if hasattr(y, 'values') else y

        X_tensor = torch.tensor(X_np, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y_np, dtype=torch.float32).view(-1, 1).to(self.device)
        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for epoch in range(self.epochs):
            for xb, yb in loader:
                out = self.model(xb)
                loss = self.loss_fn(out, yb)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
        return self

    def predict_proba(self, X):
        self.model.eval()
        with torch.no_grad():
            X_np = X.values if hasattr(X, 'values') else X
            X_tensor = torch.tensor(X_np, dtype=torch.float32).to(self.device)
            logits = self.model(X_tensor)
            probs = torch.sigmoid(logits).cpu().numpy()
        return np.hstack([1 - probs, probs])

    def predict(self, X):
        probs = self.predict_proba(X)
        return (probs[:, 1] >= 0.5).astype(int)

In [26]:
methods = [
    'Base', 'Undersampling', 'SMOTE-NC',
    'table-gan', 'Smotified-tablegan', 'vae-tablegan',
    'ctgan', 'smotified-ctgan', 'vae-ctgan',
    'smotified-vae-ctgan', 'ctabgan', 'smotified-ctabgan', 'vae-ctabgan', 'smotified-vae-ctabgan'
]

X_test = test_classification.drop(columns='loan_status')
y_test = test_classification['loan_status']

returns_df = pd.DataFrame()

for method in tqdm(methods,desc="Methods"):
    data_x = data_classification.drop(columns='loan_status')
    data_y = data_classification['loan_status']
    input_dim = data_x.shape[1]

    model = MLPClassifierTorch(
        input_dim=input_dim,
        hidden_dims=(64, 32),   
        lr=0.001,
        epochs=20,
        batch_size=256,
        device='cuda'            
    )
    
    return_evaluation(model, data_x, data_y, X_test, y_test, method)

#classification 결과
classification_metrics_df = pd.DataFrame(classification_metrics_summary)

#상위 10% 투자전략
returns_with_sharpe_df_top = pd.DataFrame(summary_with_sharpe_top)
portfolio_grade_distribution_df_top = pd.DataFrame(summary_with_grade_top)

Methods:   0%|          | 0/14 [00:00<?, ?it/s]

Preprocessing method : Base


Methods:   0%|          | 0/14 [02:42<?, ?it/s]


MemoryError: Unable to allocate 290. MiB for an array with shape (1116458, 34) and data type float64

In [ ]:
classification_metrics_df.head(15)

,Method,Class 0 Precision,Class 0 Recall,Class 0 F1,Class 1 Precision,Class 1 Recall,Class 1 F1,Macro Avg F1,Weighted Avg F1
0,Base,0.942723,0.947631,0.945171,0.779163,0.762425,0.770703,0.857937,0.911137
1,Undersampling,0.975910,0.878015,0.924378,0.644001,0.910567,0.754430,0.839404,0.891226
2,SMOTE-NC,0.958913,0.921593,0.939882,0.721232,0.837057,0.774840,0.857361,0.907687
3,table-gan,0.945181,0.939554,0.942359,0.756559,0.775146,0.765740,0.854050,0.907906
4,Smotified-tablegan,0.942790,0.947614,0.945196,0.779175,0.762721,0.770860,0.858028,0.911188
5,vae-tablegan,0.942760,0.947626,0.945187,0.779184,0.762590,0.770798,0.857992,0.911169
6,ctgan,0.965962,0.903064,0.933454,0.684717,0.868691,0.765810,0.849632,0.900752
7,smotified-ctgan,0.956461,0.926550,0.941268,0.731559,0.825961,0.775899,0.858584,0.909009
8,vae-ctgan,0.942781,0.947604,0.945187,0.779134,0.762687,0.770823,0.858005,0.911173
9,smotified-vae-ctgan,0.942750,0.947636,0.945187,0.779206,0.762542,0.770784,0.857985,0.911166


In [ ]:
returns_with_sharpe_df_top.head(15)

,Method,Average Return,Std Dev,Sharpe Ratio
0,Base,19.573532,0.158354,1.236065
1,Undersampling,15.333564,0.128174,1.196307
2,SMOTE-NC,15.021282,0.121848,1.232785
3,table-gan,14.347661,0.111915,1.282013
4,Smotified-tablegan,17.024341,0.129161,1.318071
5,vae-tablegan,17.024265,0.129160,1.318078
6,ctgan,15.276062,0.117074,1.304824
7,smotified-ctgan,15.070654,0.113863,1.323582
8,vae-ctgan,17.024410,0.129162,1.318067
9,smotified-vae-ctgan,17.022362,0.129227,1.317244


In [ ]:
portfolio_grade_distribution_df_top.head(15)

,Method,A,B,C,D,E,F+G,Total
0,Base,7574,15503,17370,10098,6377,3301,60223
1,Undersampling,16009,16710,12376,5233,2518,1055,53901
2,SMOTE-NC,18213,18018,12765,5570,2184,829,57579
3,table-gan,16048,18982,14271,7126,2404,723,59554
4,Smotified-tablegan,12876,18751,15920,7967,3363,1341,60218
5,vae-tablegan,12877,18751,15921,7967,3363,1341,60220
6,ctgan,16590,18429,12983,5293,2019,696,56010
7,smotified-ctgan,17995,19083,12936,5413,1958,652,58037
8,vae-ctgan,12876,18750,15920,7967,3363,1341,60217
9,smotified-vae-ctgan,12876,18751,15921,7968,3363,1342,60221


In [ ]:
portfolio_grade = portfolio_grade_distribution_df_top.copy()

grade_columns = ['A', 'B', 'C', 'D', 'E', 'F+G']

for col in grade_columns:
    portfolio_grade[col] = (portfolio_grade[col] / portfolio_grade['Total']) * 100
    portfolio_grade[col] = round(portfolio_grade[col],2)

# 결과 출력
print(portfolio_grade[[col for col in grade_columns]])


        A      B      C      D      E   F+G
0   12.58  25.74  28.84  16.77  10.59  5.48
1   29.70  31.00  22.96   9.71   4.67  1.96
2   31.63  31.29  22.17   9.67   3.79  1.44
3   26.95  31.87  23.96  11.97   4.04  1.21
4   21.38  31.14  26.44  13.23   5.58  2.23
5   21.38  31.14  26.44  13.23   5.58  2.23
6   29.62  32.90  23.18   9.45   3.60  1.24
7   31.01  32.88  22.29   9.33   3.37  1.12
8   21.38  31.14  26.44  13.23   5.58  2.23
9   21.38  31.14  26.44  13.23   5.58  2.23
10   3.61  18.61  33.83  27.47  11.61  4.87
11  67.30  24.40   6.78   1.32   0.19  0.02
12  25.93  30.02  24.50  12.15   5.25  2.14
13  39.54  30.42  18.58   7.69   2.79  0.98


In [ ]:
portfolio_grade.head(15)

,Method,A,B,C,D,E,F+G,Total
0,Base,12.58,25.74,28.84,16.77,10.59,5.48,60223
1,Undersampling,29.70,31.00,22.96,9.71,4.67,1.96,53901
2,SMOTE-NC,31.63,31.29,22.17,9.67,3.79,1.44,57579
3,table-gan,26.95,31.87,23.96,11.97,4.04,1.21,59554
4,Smotified-tablegan,21.38,31.14,26.44,13.23,5.58,2.23,60218
5,vae-tablegan,21.38,31.14,26.44,13.23,5.58,2.23,60220
6,ctgan,29.62,32.90,23.18,9.45,3.60,1.24,56010
7,smotified-ctgan,31.01,32.88,22.29,9.33,3.37,1.12,58037
8,vae-ctgan,21.38,31.14,26.44,13.23,5.58,2.23,60217
9,smotified-vae-ctgan,21.38,31.14,26.44,13.23,5.58,2.23,60221
